In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta,date
import matplotlib.pyplot as plt
import folium
import plotly.express as px
from matplotlib import ticker 
from scipy.interpolate import make_interp_spline, BSpline
import pycountry_convert as pc
#import geopandas as gpd

## Interactive web-based dashboard https://www.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6

## COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University
### https://github.com/CSSEGISandData/COVID-19/

### Get Data

In [3]:
# Retriving Dataset from 
# https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data
# which is updated daily

url_confirmed = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
df_confirmed = pd.read_csv( url_confirmed )

url_death = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
df_deaths = pd.read_csv( url_death )

url_recovered = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"
df_recovered = pd.read_csv(url_recovered)
 
#url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv"
#df_confirmed_US = pd.read_csv( url )

In [4]:
df_confirmed.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/20/20,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,44443,44503,44706,44988,45280,45490,45716,45839,45966,46215
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,31459,32196,32761,33556,34300,34944,35600,36245,36790,37625
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,72755,73774,74862,75867,77000,78025,79110,80168,81212,82221
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,6142,6207,6256,6304,6351,6428,6534,6610,6610,6712
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,14267,14413,14493,14634,14742,14821,14920,15008,15087,15103


In [5]:
# Check the size of the each of the three dataframe
print(df_confirmed.shape)
print(df_deaths.shape)
print(df_recovered.shape)

(271, 317)
(271, 317)
(256, 317)


In [6]:
# The recovered dataframe has less rows, because there is only one row for Canada, no data on Province/State level
df_recovered[df_recovered["Country/Region"]=='Canada']

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/20/20,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20
39,NaN,Canada,56.1304,-106.3468,0,0,0,0,0,0,...,261754,265568,269137,274315,278191,281883,285672,291221,295475,299095


In [7]:
# Transform from wide to long format table
df_confirmed = pd.melt(df_confirmed, id_vars=df_confirmed.columns[:4], 
                    value_vars = df_confirmed.columns[4:], 
                    var_name = 'date', 
                    value_name = 'confirmed')
df_confirmed.head()

,Province/State,Country/Region,Lat,Long,date,confirmed
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0
1,NaN,Albania,41.15330,20.168300,1/22/20,0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0


In [8]:
# Transform from wide to long format table
df_deaths = pd.melt(df_deaths, id_vars=df_deaths.columns[:4], 
                    value_vars = df_deaths.columns[4:], 
                    var_name = 'date', 
                    value_name = 'deaths')

In [9]:
# Transform from wide to long format table
df_recovered = pd.melt(df_recovered, id_vars=df_recovered.columns[:4], 
                    value_vars = df_recovered.columns[4:], 
                    var_name = 'date', 
                    value_name = 'recovered')

In [10]:
# Check the size of the each of the three dataframe
print(df_confirmed.shape)
print(df_deaths.shape)
print(df_recovered.shape)

(84823, 6)
(84823, 6)
(80128, 6)


In [11]:
# Summarize confirmed and deaths data by date
df_confirmed_canada = df_confirmed[df_confirmed['Country/Region'] == 'Canada'].groupby('date').sum()[['confirmed']]
df_deaths_canada = df_deaths[df_deaths['Country/Region'] == 'Canada'].groupby('date').sum()[['deaths']]

# Extract columns from recoveries table (drop the last deaths value column)
df_recovered_canada = df_recovered[df_recovered['Country/Region'] == 'Canada']
df_canada_template  = df_recovered_canada[df_recovered_canada.columns[:-1]].reset_index(drop=True)

# Join aggrregated confirmed and deaths data with extracted columns
df_confirmed_canada = df_canada_template.merge(df_confirmed_canada, how='inner', left_on='date', right_index=True)
df_deaths_canada = df_canada_template.merge(df_deaths_canada, how='inner', left_on='date', right_index=True)

# Add the agrregated data for Canada back to confirmed and deaths table
df_confirmed = df_confirmed[df_confirmed['Country/Region'] != 'Canada'].append(df_confirmed_canada)
df_deaths = df_deaths[df_deaths['Country/Region'] != 'Canada'].append(df_deaths_canada)

In [12]:
df_recovered.head()

,Province/State,Country/Region,Lat,Long,date,recovered
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0
1,NaN,Albania,41.15330,20.168300,1/22/20,0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0


In [13]:
# Merge the three table together
df_data = df_confirmed.merge(df_deaths, how='inner', on=['Country/Region','Province/State','date'])
df_data = df_data.merge(df_recovered, how='inner', on=['Country/Region','Province/State','date'])

In [14]:
df_data.head()

,Province/State,Country/Region,Lat_x,Long_x,date,confirmed,Lat_y,Long_y,deaths,Lat,Long,recovered
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0,33.93911,67.709953,0,33.93911,67.709953,0
1,NaN,Albania,41.15330,20.168300,1/22/20,0,41.15330,20.168300,0,41.15330,20.168300,0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0,28.03390,1.659600,0,28.03390,1.659600,0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0,42.50630,1.521800,0,42.50630,1.521800,0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0,-11.20270,17.873900,0,-11.20270,17.873900,0


In [15]:
# Drop the columns 
df_data = df_data.drop(['Lat_x','Lat_y','Long_x','Long_y'],axis=1)

In [16]:
df_data.head()

,Province/State,Country/Region,date,confirmed,deaths,Lat,Long,recovered
0,NaN,Afghanistan,1/22/20,0,0,33.93911,67.709953,0
1,NaN,Albania,1/22/20,0,0,41.15330,20.168300,0
2,NaN,Algeria,1/22/20,0,0,28.03390,1.659600,0
3,NaN,Andorra,1/22/20,0,0,42.50630,1.521800,0
4,NaN,Angola,1/22/20,0,0,-11.20270,17.873900,0


In [17]:
# Get the population of the county data
url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv"
df_pop = pd.read_csv( url )

In [18]:
df_pop.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population
0,4,AF,AFG,4.0,NaN,NaN,NaN,Afghanistan,33.93911,67.709953,Afghanistan,38928341.0
1,8,AL,ALB,8.0,NaN,NaN,NaN,Albania,41.15330,20.168300,Albania,2877800.0
2,12,DZ,DZA,12.0,NaN,NaN,NaN,Algeria,28.03390,1.659600,Algeria,43851043.0
3,20,AD,AND,20.0,NaN,NaN,NaN,Andorra,42.50630,1.521800,Andorra,77265.0
4,24,AO,AGO,24.0,NaN,NaN,NaN,Angola,-11.20270,17.873900,Angola,32866268.0


In [19]:
# Check if the name of Country matchs in two tables
for c in df_data['Country/Region'].unique():
  if c not in df_pop['Country_Region'].unique():
    print(c)

In [20]:
# rename the column name
df_data = df_data.rename(columns={"Country/Region":"Country_Region","Province/State": "Province_State"})

In [21]:
df_data1 = df_data.merge(df_pop[['Country_Region','Province_State','Population']], how='inner', on=['Country_Region','Province_State'])

In [22]:
df_data1[df_data1.Population.isnull()].Country_Region

27544    Diamond Princess
27545    Diamond Princess
27546    Diamond Princess
27547    Diamond Princess
27548    Diamond Princess
               ...       
48510          MS Zaandam
48511          MS Zaandam
48512          MS Zaandam
48513          MS Zaandam
48514          MS Zaandam
Name: Country_Region, Length: 626, dtype: object

In [23]:
df_data1.to_csv('covid19_confirmed_deaths_recovered_global.csv',index=False)

In [24]:
test = pd.read_csv('covid19_confirmed_deaths_recovered_global.csv' )

In [25]:
test.head()

,Province_State,Country_Region,date,confirmed,deaths,Lat,Long,recovered,Population
0,NaN,Afghanistan,1/22/20,0,0,33.93911,67.709953,0,38928341.0
1,NaN,Afghanistan,1/23/20,0,0,33.93911,67.709953,0,38928341.0
2,NaN,Afghanistan,1/24/20,0,0,33.93911,67.709953,0,38928341.0
3,NaN,Afghanistan,1/25/20,0,0,33.93911,67.709953,0,38928341.0
4,NaN,Afghanistan,1/26/20,0,0,33.93911,67.709953,0,38928341.0
